# Motor de Correspondência - Blocking e Algoritmos de Similaridade

Este notebook implementa o motor de correspondência que encontra matches entre endereços normalizados e a Camada Ouro usando estratégias de blocking e algoritmos de similaridade.

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re
import math

# Configurações
THRESHOLD_SIMILARIDADE = 0.75  # Threshold mínimo para considerar match
THRESHOLD_ALTA_CONFIANCA = 0.90  # Threshold para alta confiança

In [ ]:
# UDF para SoundexBR (simplificado)
def soundex_br(texto):
    """Gera código SoundexBR para comparação fonética"""
    if not texto:
        return None
    
    texto = texto.upper().strip()
    
    # Mapeamento de consoantes para dígitos
    mapa = {
        'B': '1', 'F': '1', 'P': '1', 'V': '1',
        'C': '2', 'G': '2', 'J': '2', 'K': '2', 'Q': '2', 'S': '2', 'X': '2', 'Z': '2',
        'D': '3', 'T': '3',
        'L': '4',
        'M': '5', 'N': '5',
        'R': '6'
    }
    
    # Primeira letra
    codigo = texto[0] if texto else ''
    
    # Converter consoantes
    for char in texto[1:]:
        if char in mapa:
            digito = mapa[char]
            if codigo[-1] != digito:
                codigo += digito
    
    # Completar com zeros
    codigo = codigo[:4].ljust(4, '0')
    
    return codigo

udf_soundex_br = udf(soundex_br, StringType())

In [ ]:
# UDF para Distância de Levenshtein
def levenshtein_distance(s1, s2):
    """Calcula distância de Levenshtein entre duas strings"""
    if not s1 or not s2:
        return max(len(s1 or ''), len(s2 or ''))
    
    s1 = s1.upper()
    s2 = s2.upper()
    
    if s1 == s2:
        return 0
    
    len_s1 = len(s1)
    len_s2 = len(s2)
    
    # Matriz de distâncias
    d = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]
    
    # Inicializar primeira linha e coluna
    for i in range(len_s1 + 1):
        d[i][0] = i
    for j in range(len_s2 + 1):
        d[0][j] = j
    
    # Preencher matriz
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = 1
            
            d[i][j] = min(
                d[i-1][j] + 1,      # Deletar
                d[i][j-1] + 1,      # Inserir
                d[i-1][j-1] + cost  # Substituir
            )
    
    return d[len_s1][len_s2]

def levenshtein_similarity(s1, s2):
    """Calcula similaridade baseada em Levenshtein (0-1)"""
    if not s1 or not s2:
        return 0.0
    
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    
    distance = levenshtein_distance(s1, s2)
    similarity = 1.0 - (distance / max_len)
    
    return similarity

udf_levenshtein_sim = udf(levenshtein_similarity, DoubleType())

In [ ]:
# UDF para Jaro-Winkler (reutilizado do notebook de clusterização)
def jaro_winkler_similarity(s1, s2):
    """Calcula similaridade Jaro-Winkler"""
    if not s1 or not s2:
        return 0.0
    
    s1 = s1.upper().strip()
    s2 = s2.upper().strip()
    
    if s1 == s2:
        return 1.0
    
    len_s1 = len(s1)
    len_s2 = len(s2)
    match_window = max(len_s1, len_s2) // 2 - 1
    if match_window < 0:
        match_window = 0
    
    s1_matches = [False] * len_s1
    s2_matches = [False] * len_s2
    matches = 0
    transpositions = 0
    
    for i in range(len_s1):
        start = max(0, i - match_window)
        end = min(i + match_window + 1, len_s2)
        
        for j in range(start, end):
            if s2_matches[j] or s1[i] != s2[j]:
                continue
            s1_matches[i] = True
            s2_matches[j] = True
            matches += 1
            break
    
    if matches == 0:
        return 0.0
    
    k = 0
    for i in range(len_s1):
        if not s1_matches[i]:
            continue
        while not s2_matches[k]:
            k += 1
        if s1[i] != s2[k]:
            transpositions += 1
        k += 1
    
    jaro = (matches / len_s1 + matches / len_s2 + (matches - transpositions / 2) / matches) / 3.0
    
    prefix = 0
    for i in range(min(len(s1), len(s2), 4)):
        if s1[i] == s2[i]:
            prefix += 1
        else:
            break
    
    winkler = jaro + (0.1 * prefix * (1 - jaro))
    
    return min(1.0, winkler)

udf_jaro_winkler = udf(jaro_winkler_similarity, DoubleType())

In [ ]:
# Função de blocking: Filtro Rígido (UF + Cidade)
def aplicar_blocking_rigido(df_enderecos, df_camada_ouro):
    """
    Aplica blocking rígido: filtra apenas endereços da mesma UF e Cidade
    
    Args:
        df_enderecos: DataFrame com endereços normalizados
        df_camada_ouro: DataFrame com Camada Ouro
    
    Returns:
        DataFrame com candidatos após blocking rígido
    """
    df_candidatos = df_enderecos.alias("e").join(
        df_camada_ouro.alias("o"),
        (col("e.uf") == col("o.uf")) & (col("e.cidade") == col("o.cidade")),
        "inner"
    )
    
    return df_candidatos

In [ ]:
# Função de blocking: Filtro Flexível (Fonética do Bairro)
def aplicar_blocking_flexivel(df_candidatos_rigido):
    """
    Aplica blocking flexível usando SoundexBR do bairro
    
    Args:
        df_candidatos_rigido: DataFrame após blocking rígido
    
    Returns:
        DataFrame com candidatos após blocking flexível
    """
    df_candidatos_flex = df_candidatos_rigido \
        .withColumn("soundex_bairro_e", udf_soundex_br(col("e.bairro_norm"))) \
        .withColumn("soundex_bairro_o", udf_soundex_br(col("o.bairro"))) \
        .filter(
            (col("soundex_bairro_e") == col("soundex_bairro_o")) |
            (col("e.bairro_norm").isNull()) |
            (col("o.bairro").isNull())
        )
    
    return df_candidatos_flex

In [ ]:
# Função para calcular scores de similaridade
def calcular_scores_similaridade(df_candidatos):
    """
    Calcula múltiplos scores de similaridade para cada candidato
    
    Args:
        df_candidatos: DataFrame com candidatos após blocking
    
    Returns:
        DataFrame com scores de similaridade
    """
    df_com_scores = df_candidatos \
        .withColumn("sim_jaro_winkler", 
            udf_jaro_winkler(col("e.nome_logradouro_norm"), col("o.nome_logradouro"))) \
        .withColumn("sim_levenshtein", 
            udf_levenshtein_sim(col("e.nome_logradouro_norm"), col("o.nome_logradouro"))) \
        .withColumn("match_numero", 
            when(col("e.numero") == col("o.numero"), 1.0).otherwise(0.0)) \
        .withColumn("match_tipo_log", 
            when(col("e.tipo_logradouro_norm") == col("o.tipo_logradouro"), 1.0).otherwise(0.0))
    
    # Score combinado (pesos configuráveis)
    df_com_scores = df_com_scores \
        .withColumn("score_final", 
            (col("sim_jaro_winkler") * 0.4 + 
             col("sim_levenshtein") * 0.3 + 
             col("match_numero") * 0.2 + 
             col("match_tipo_log") * 0.1)
        ) \
        .withColumn("confianca", 
            when(col("score_final") >= THRESHOLD_ALTA_CONFIANCA, "ALTA")
            .when(col("score_final") >= THRESHOLD_SIMILARIDADE, "MEDIA")
            .otherwise("BAIXA")
        )
    
    return df_com_scores

In [ ]:
# Função principal do motor de correspondência
def executar_motor_correspondencia(df_enderecos_normalizados):
    """
    Executa motor de correspondência completo
    
    Args:
        df_enderecos_normalizados: DataFrame com endereços normalizados
    
    Returns:
        DataFrame com matches encontrados
    """
    print("1. Carregando Camada Ouro...")
    df_camada_ouro = read_delta_table(PATH_CAMADA_OURO)
    print(f"   Registros na Camada Ouro: {df_camada_ouro.count()}")
    
    print("2. Aplicando blocking rígido (UF + Cidade)...")
    df_candidatos_rigido = aplicar_blocking_rigido(df_enderecos_normalizados, df_camada_ouro)
    print(f"   Candidatos após blocking rígido: {df_candidatos_rigido.count()}")
    
    print("3. Aplicando blocking flexível (SoundexBR bairro)...")
    df_candidatos_flex = aplicar_blocking_flexivel(df_candidatos_rigido)
    print(f"   Candidatos após blocking flexível: {df_candidatos_flex.count()}")
    
    print("4. Calculando scores de similaridade...")
    df_com_scores = calcular_scores_similaridade(df_candidatos_flex)
    
    print("5. Filtrando matches acima do threshold...")
    df_matches = df_com_scores \
        .filter(col("score_final") >= THRESHOLD_SIMILARIDADE) \
        .select(
            col("e.id").alias("id_endereco_input"),
            col("e.endereco_livre"),
            col("e.endereco_normalizado"),
            col("o.uid").alias("uid_camada_ouro"),
            col("o.endereco_normalizado").alias("endereco_camada_ouro"),
            col("sim_jaro_winkler"),
            col("sim_levenshtein"),
            col("score_final"),
            col("confianca"),
            col("o.latitude"),
            col("o.longitude"),
            col("o.score_confianca").alias("score_confianca_ouro")
        )
    
    # Para cada endereço de entrada, manter apenas o melhor match
    from pyspark.sql.window import Window
    window_spec = Window.partitionBy("id_endereco_input").orderBy(desc("score_final"))
    
    df_matches_final = df_matches \
        .withColumn("rank", row_number().over(window_spec)) \
        .filter(col("rank") == 1) \
        .drop("rank") \
        .withColumn("match_em", current_timestamp())
    
    print(f"   Matches encontrados: {df_matches_final.count()}")
    
    return df_matches_final

In [ ]:
# Exemplo de uso
# df_enderecos_normalizados = read_delta_table(PATH_ENDERECOS_NORMALIZADOS)
# df_matches = executar_motor_correspondencia(df_enderecos_normalizados)

# Visualizar matches
# df_matches.show(20, truncate=False)

# Estatísticas
# df_matches.groupBy("confianca").agg(
#     count("*").alias("quantidade")
# ).show()

In [ ]:
# Salvar matches
# save_delta_table(df_matches, PATH_MATCHES, mode="append", partition_by=["confianca"])

print("Motor de correspondência executado com sucesso!")